## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [18]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        # 声明超参数
        self.input_dim = 28 * 28 # 输入形状
        self.hidden_dim = 2048 # 隐藏层单元数
        self.output_dim = 10 # 输出形状
        
        # 声明模型对应的参数
        self.W1 = tf.Variable(tf.random.normal([self.input_dim, self.hidden_dim], stddev=0.01), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([self.hidden_dim]), dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([self.hidden_dim, self.output_dim], stddev=0.01), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([self.output_dim]), dtype=tf.float32)
        ####################

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = tf.reshape(x, [-1, self.input_dim])  # 输入数据展平
        h1 = tf.matmul(x, self.W1) + self.b1  # 第一层线性变换
        x = tf.tanh(h1)  # 激活函数
        logits = tf.matmul(x, self.W2) + self.b2  # 第二层线性变换        
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [12]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [19]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3108344 ; accuracy 0.06576667
epoch 1 : loss 2.3064072 ; accuracy 0.07425
epoch 2 : loss 2.3019998 ; accuracy 0.08846667
epoch 3 : loss 2.297611 ; accuracy 0.10723333
epoch 4 : loss 2.29324 ; accuracy 0.13148333
epoch 5 : loss 2.2888865 ; accuracy 0.15696667
epoch 6 : loss 2.2845502 ; accuracy 0.18383333
epoch 7 : loss 2.2802303 ; accuracy 0.21148333
epoch 8 : loss 2.2759256 ; accuracy 0.23861666
epoch 9 : loss 2.2716365 ; accuracy 0.26581666
epoch 10 : loss 2.2673619 ; accuracy 0.29265
epoch 11 : loss 2.2631016 ; accuracy 0.31896666
epoch 12 : loss 2.258855 ; accuracy 0.34526667
epoch 13 : loss 2.254622 ; accuracy 0.37135
epoch 14 : loss 2.2504015 ; accuracy 0.39611667
epoch 15 : loss 2.2461932 ; accuracy 0.42003334
epoch 16 : loss 2.2419968 ; accuracy 0.441
epoch 17 : loss 2.2378123 ; accuracy 0.46115
epoch 18 : loss 2.2336385 ; accuracy 0.47888333
epoch 19 : loss 2.2294753 ; accuracy 0.49523333
epoch 20 : loss 2.2253225 ; accuracy 0.5101167
epoch 21 : loss 2.2211792